Using a dataset of your choice, select an outcome variable and then pick four or five other variables (one to two categorical, three to four continuous) to act as the basis for features. Explore the variables using the univariate and bivariate methods you've learned so far. 

Next, based on what you learned via your data exploration, create ten new features. Explain the reasoning behind each one.

Finally, use filtering methods to select the five best features and justify your choices.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import csv, sqlite3
%matplotlib inline

from GDELT_utils import GDELT_columns, usecols, dtype_dict, \
                        cameo_dict, map_cameo_to_text, \
                        state_dict, mem_usage, state_heat_map

In [2]:
con = sqlite3.connect("gdelt.db")

In [3]:
#c = con.cursor()
#c.execute("CREATE INDEX idx_SQLDATE ON gdelt (SQLDATE)")
#con.commit()

OperationalError: index idx_SQLDATE already exists

In [4]:
for row in c.execute("EXPLAIN QUERY PLAN \
           SELECT SQLDATE, Actor1CountryCode, Actor2CountryCode, \
                            Actor1Geo_CountryCode, Actor2Geo_CountryCode, \
                            NumMentions, EventRootCode, AvgTone \
           FROM gdelt \
           WHERE SQLDATE > \"2017-05-01\" AND SQLDATE < \"2017-08-01\" \
           LIMIT 100"):
    print(row)

(0, 0, 0, 'SEARCH TABLE gdelt USING INDEX idx_SQLDATE (SQLDATE>? AND SQLDATE<?)')


In [15]:
gdelt = pd.read_sql_query("SELECT SQLDATE, Actor1CountryCode, Actor2CountryCode, \
                            Actor1Geo_CountryCode, Actor2Geo_CountryCode, \
                            EventRootCode, AVG(NumMentions), AVG(AvgTone) \
                          FROM gdelt \
                          WHERE SQLDATE > \"2017-05-01\" AND SQLDATE < \"2017-08-01\" \
                          GROUP BY SQLDATE, Actor1CountryCode, Actor2CountryCode, \
                          Actor1Geo_CountryCode, Actor2Geo_CountryCode, EventRootCode", con)
# aggregate data by
# Date Actor1Code Actor2Code Actor1Geo_CountryCode Actor2Geo_CountryCode EventRootCode -> Mean AvgTone

# Features Actor1Code Actor2Code Actor1Geo_CountryCode Actor2Geo_CountryCode NumMentions EventRootCode 

In [ ]:
interest = ["SQLDATE",
            "AvgTone", "NumMentions", "EventRootCode",
            "Actor1CountryCode", "Actor2CountryCode",
            "Actor1Geo_CountryCode", "Actor2Geo_CountryCode"]

categories = ["EventRootCode",
            "Actor1CountryCode", "Actor2CountryCode",
            "Actor1Geo_CountryCode", "Actor2Geo_CountryCode"]

for category_col in categories:
    gdelt[category_col] = gdelt[category_col].astype('category')
    
gdelt['SQLDATE'] = pd.to_datetime(gdelt['SQLDATE'])  

In [16]:
gdelt

,SQLDATE,Actor1CountryCode,Actor2CountryCode,Actor1Geo_CountryCode,Actor2Geo_CountryCode,EventRootCode,AVG(NumMentions),AVG(AvgTone)
0,2017-05-01 00:00:00,None,None,None,None,01,60.184049,-2.723992
1,2017-05-01 00:00:00,None,None,None,None,02,76.748634,-1.124452
2,2017-05-01 00:00:00,None,None,None,None,03,35.639269,0.135407
3,2017-05-01 00:00:00,None,None,None,None,04,62.144420,-0.913240
4,2017-05-01 00:00:00,None,None,None,None,05,39.661017,1.502765
5,2017-05-01 00:00:00,None,None,None,None,06,37.456140,-1.328068
6,2017-05-01 00:00:00,None,None,None,None,07,34.626168,-0.422873
7,2017-05-01 00:00:00,None,None,None,None,08,32.846715,-2.303627
8,2017-05-01 00:00:00,None,None,None,None,09,122.980769,-3.491739
9,2017-05-01 00:00:00,None,None,None,None,10,41.666667,-2.072871


In [23]:
gdelt.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4370491 entries, 0 to 4370490
Data columns (total 8 columns):
SQLDATE                  datetime64[ns]
Actor1CountryCode        category
Actor2CountryCode        category
Actor1Geo_CountryCode    category
Actor2Geo_CountryCode    category
EventRootCode            category
AVG(NumMentions)         float64
AVG(AvgTone)             float64
dtypes: category(5), datetime64[ns](1), float64(2)
memory usage: 137.6 MB


In [93]:
gdelt_sample = gdelt.sample(frac=.2)

In [94]:
# sample on the dataframe / don't use all possible data available

# standardize data around mean 0 

# NormNumMentions -> normalized for the day
# NormNumMentions -> normalized by rolling average
# NormNumMentions -> normalized by the year
# AvgTone + NumMentions

# split data in missing/non-missing and compare distributions of that data

# which features are generally missing record values

# Try to predict AvgTone 

In [95]:
gdelt_sample['Actor1CountryCode'].value_counts()[gdelt_sample['Actor1CountryCode'].value_counts()/len(gdelt) > .001] 

USA    76399
GBR    26972
CHN    22948
RUS    22400
FRA    16162
EUR    16021
DEU    14739
ISR    12728
SAU    12399
AUS    11926
TUR    11424
IRN    11370
CAN    11352
SYR    10169
AFR     9801
JPN     9336
PAK     8959
ITA     8321
QAT     8066
EGY     7252
PRK     7199
IND     6850
PHL     6516
KOR     6399
IRQ     6122
AFG     5982
PSE     5808
ESP     5732
NGA     5645
IRL     5321
JOR     5128
BEL     4929
MEX     4854
ARE     4686
MYS     4618
POL     4474
UKR     4468
Name: Actor1CountryCode, dtype: int64

In [118]:
def unify_rare_cats(data, category, cut_off):
    vc = data[category].value_counts()
    print(vc)
    past_cut_off = vc/len(data) > cut_off
    remaining = vc[past_cut_off].index
    print('ZWE' in remaining)
    print(data[category])
    data[(category + '_unify')] = data[category].apply(lambda x: if x in remaining else "OTHER", axis=1)

SyntaxError: invalid syntax (<ipython-input-118-e10a3c1887f1>, line 8)

In [119]:
unify_rare_cats(gdelt_sample, 'Actor1CountryCode', .001)
unify_rare_cats(gdelt_sample, 'Actor2CountryCode', .001)
unify_rare_cats(gdelt_sample, 'Actor1Geo_CountryCode', .001)
unify_rare_cats(gdelt_sample, 'Actor2Geo_CountryCode', .001)

USA    76399
GBR    26972
CHN    22948
RUS    22400
FRA    16162
EUR    16021
DEU    14739
ISR    12728
SAU    12399
AUS    11926
TUR    11424
IRN    11370
CAN    11352
SYR    10169
AFR     9801
JPN     9336
PAK     8959
ITA     8321
QAT     8066
EGY     7252
PRK     7199
IND     6850
PHL     6516
KOR     6399
IRQ     6122
AFG     5982
PSE     5808
ESP     5732
NGA     5645
IRL     5321
       ...  
NRU      110
SAF      109
DMA      105
GNB      100
CPV       89
KIR       86
CAS       84
SUR       77
CRB       66
ABW       65
LIE       64
COM       60
AIA       59
PLW       56
MAC       55
FSM       53
TUV       37
EAF       31
STP       27
SMR       27
GEO       17
AND       11
SHN       11
LAM        3
WLF        2
SAM        2
ROM        1
CAU        1
SCN        1
PRI        0
Name: Actor1CountryCode, Length: 221, dtype: int64
True
3757494    NaN
996344     ESP
1652706    NaN
4258603    NaN
1865696    MOZ
2977498    USA
3349537    NaN
2224066    USA
3315506    NaN
193270     LBY
2

TypeError: <lambda>() got an unexpected keyword argument 'axis'

In [101]:
gdelt_sample['Actor1CountryCode_unify'].value_counts()

ZWE      256334
USA       76399
OTHER     39004
GBR       26972
CHN       22948
RUS       22400
FRA       16162
EUR       16021
DEU       14739
ISR       12728
SAU       12399
AUS       11926
TUR       11424
IRN       11370
CAN       11352
SYR       10169
AFR        9801
JPN        9336
PAK        8959
ITA        8321
QAT        8066
EGY        7252
PRK        7199
IND        6850
PHL        6516
KOR        6399
IRQ        6122
AFG        5982
PSE        5808
ESP        5732
          ...  
CZE        1654
NPL        1642
TZA        1635
KHM        1612
PAN        1588
JAM        1580
CHL        1467
PER        1465
CYP        1434
TUN        1420
BGR        1325
MCO        1305
NMR        1298
OMN        1260
ZMB        1209
RWA        1104
MLT        1098
ECU        1097
DZA        1060
TCD        1048
EST        1047
LTU        1006
FJI         998
LUX         978
COD         967
LBR         947
MLI         924
SRB         887
BHS         882
GUY         878
Name: Actor1CountryCode_

In [ ]:
gdelt_sample['Actor1CountryCode-cut'] = gdelt_sample['Actor1CountryCode'].apply(lambda x: )

In [26]:
pd.get_dummies(gdelt_sample['Actor1CountryCode'])

,ABW,AFG,AFR,AGO,AIA,ALB,AND,ARE,ARG,ARM,...,VNM,VUT,WAF,WLF,WSM,WST,YEM,ZAF,ZMB,ZWE
3763975,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2372840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3532814,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
706616,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2748387,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2654113,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3056998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2594242,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3045918,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996616,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
